In [1]:
import sys
sys.path.append('../scared_lbc')

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
import cupy as _cu
from tqdm.auto import tqdm
import numpy as np
import math

In [4]:
import scared_cu

In [5]:
reduction = scared_cu.lbc.modop.Reduction_Q2Q2(3329, 'int32')

model = scared_cu.models.AbsoluteValue()


N = 2000
sigma = 0
conv = 250

# scared_cu.container.set_batch_size(5)

n = 100 #don't touch
n_step = 1

neg_trick = False

prng_off = False

incomplete = 0

cu_step = 1 << 10

In [6]:
def hocpa_wrapper(ths, reduction, model, step=1000, conv_step=1000, neg_trick=False, subkeys=None, early_stop=True, pp=True):


    selection_function = scared_cu.lbc.selection_functions.BaseMulLargeQ(reduction=reduction, words=subkeys, cu_step=cu_step)

    selection_function.reset()

    for _ in tqdm(range(selection_function.num_steps)):

        assert selection_function.done() == False


        cpa = scared_cu.CPAAttackAlternative(selection_function=selection_function,
                                                model=model,
                                                discriminant=scared_cu.maxabs,
                                                convergence_step=conv_step)


        mean = ths.samples.mean(axis=0)
        _n = ths.samples.shape[1] // 2
        pp_ = scared_cu.preprocesses.high_order.CenteredProduct(mean=mean, frame_1=range(0,_n), frame_2=range(_n,2*_n), mode='same')

        for i in range(0, len(ths), step):
            if pp:
                container = scared_cu.Container(ths[i:i + step], preprocesses=[pp_])
            else:
                container = scared_cu.Container(ths[i:i + step])
            cpa.run(container)

            # if early_stop:
            #     succs = util.succ_ratio(ths.s[0, subkeys], reduction.q, conv, cpa)
            #     if succs[-1] == 1.0:
            #         break

    selection_function.save_scores(cpa.convergence_traces)
    selection_function.next()

    return selection_function.scores

In [7]:
ths = scared_cu.lbc.simulated_traces.collect_traces_basemult(N=N, n=n, sigma=sigma, reduction=reduction, incomplete=incomplete,
                              const_seed=True, two_step=False, seed=0, prng_off=prng_off, leak_reduction=True)

print(ths)

  0%|          | 0/2000 [00:00<?, ?it/s]

Trace Header Set:
Name.............: RAM Format THS
Reader...........: RAM reader with 2000 traces. Samples shape: (2000, 200) and metadatas: ['c', 's', 's0', 's1']
c................: int32
s................: int32
s0...............: int32
s1...............: int32



In [8]:

succs = None
for n_i in tqdm(range(0, n, n_step)):
    subkeys = _cu.arange(n_i, n_i + n_step)
    assert ((ths.s % reduction.q) == ((ths.s0 + ths.s1) % reduction.q)).all()
    cpa_scores = hocpa_wrapper(ths[:N], reduction, model, step=conv, conv_step=conv, neg_trick=neg_trick, subkeys=subkeys, early_stop=False, pp=not prng_off)
    succs_i_ = scared_cu._utils.succ_ratio(ths.s[0, subkeys], reduction.q, conv, cpa_scores, incomplete=incomplete!=0, np_flag=True)
    succs_i = _cu.zeros(N//conv)
    for i in range(len(succs_i_)):
        succs_i[i] = succs_i_[i]
    for i in range(len(succs_i_), len(succs_i)):
        succs_i[i] = succs_i_[-1]
    if succs is None:
        succs = _cu.array(succs_i)
    else:
        succs += _cu.array(succs_i)
    
    for x,y in zip(range(conv,N+1,conv),succs):
        print(f'({math.log(x)/math.log(10)},{y})',end='')
    # if n_i != (n - n_step):
    #     del kw
    print()
    for i in range(n_i, n_i + n_step):
        print(f'{ths.s[0, i]}-{cpa_scores[:,i-n_i].argsort()[-1]}', end='\t')
    print()



succs /= (n//n_step)
# assert (mmia.data == (-1)).sum() == 0


  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

(2.397940008672037,1.0)(2.6989700043360183,1.0)(2.8750612633916997,1.0)(2.9999999999999996,1.0)(3.096910013008056,1.0)(3.176091259055681,1.0)(3.243038048686294,1.0)(3.301029995663981,1.0)
-586-[6 7 5 4 3 1 2 0]	


  0%|          | 0/4 [00:00<?, ?it/s]

(2.397940008672037,2.0)(2.6989700043360183,2.0)(2.8750612633916997,2.0)(2.9999999999999996,2.0)(3.096910013008056,2.0)(3.176091259055681,2.0)(3.243038048686294,2.0)(3.301029995663981,2.0)
696-[7 6 5 4 3 2 1 0]	


...

(2.397940008672037,63.0)(2.6989700043360183,63.0)(2.8750612633916997,63.0)(2.9999999999999996,63.0)(3.096910013008056,63.0)(3.176091259055681,63.0)(3.243038048686294,63.0)(3.301029995663981,63.0)
-1046-[7 6 5 3 2 4 1 0]	


  0%|          | 0/4 [00:00<?, ?it/s]

(2.397940008672037,64.0)(2.6989700043360183,64.0)(2.8750612633916997,64.0)(2.9999999999999996,64.0)(3.096910013008056,64.0)(3.176091259055681,64.0)(3.243038048686294,64.0)(3.301029995663981,64.0)
591-[7 6 4 5 2 3 1 0]	


  0%|          | 0/4 [00:00<?, ?it/s]

(2.397940008672037,65.0)(2.6989700043360183,65.0)(2.8750612633916997,65.0)(2.9999999999999996,65.0)(3.096910013008056,65.0)(3.176091259055681,65.0)(3.243038048686294,65.0)(3.301029995663981,65.0)
-667-[7 6 5 4 3 2 1 0]	


In [9]:
for x,y in zip(range(conv,N+1,conv),succs):
    print(f'({math.log(x)/math.log(10)},{y})',end='')

(2.397940008672037,0.65)(2.6989700043360183,0.65)(2.8750612633916997,0.65)(2.9999999999999996,0.65)(3.096910013008056,0.65)(3.176091259055681,0.65)(3.243038048686294,0.65)(3.301029995663981,0.65)